In [1]:
import re

In [2]:
rome_num = {
    'I': 1,
    'V': 5,
    'X': 10,
    'L': 50,
    'C': 100,
    'D': 500,
    'M': 1000
}

repeat_rules = {
    'I': 3,
    'X': 3,
    'C': 3,
    'M': 3,
    'D': 0,
    'L': 0,
    'V': 0
}

substruction_rules = {
    'I': ['V', 'X'],
    'X': ['L', 'C'],
    'C': ['D', 'M']
}

# return True if num can be substructed from num_from
def can_be_substructed(num, num_from):
    can_be_substruct_from = substruction_rules.get(num)
    if can_be_substruct_from is None:
        return False
    else:
        return num_from in can_be_substruct_from
        
        
# calc roman expressions -> arabic num
# return arabic_num, exception
def calc_roma_exp(roma_exp):
    
    exp_len = len(roma_exp)    
    total = 0
    current_idx = 0
    last_num = None
    repeat_counter = 0
    
    # go style
    try:    
        while True:

            # end
            if current_idx >= exp_len:
                break

            # current roma num & related arab num
            roma_current = roma_exp[current_idx]
            arab_current = rome_num[roma_current]

            # count repeats
            if last_num is None:
                last_num = roma_current
                repeat_counter = 1
            elif last_num == roma_current:
                repeat_counter += 1
                max_allowed_repeat = repeat_rules.get(roma_current, 0)
                if max_allowed_repeat == 0:
                    raise Exception('{} cannot be repeated'.format(roma_current))
                elif repeat_counter > max_allowed_repeat:            
                    raise Exception('{} cannot be repeated more than {} times'.format(roma_current, max_allowed_repeat))
            else:
                #reset repeted counter
                repeat_counter = 1

            # save last num
            last_num = roma_current

            # if current num is last -> sum and break
            if current_idx + 1 >= exp_len:
                total += arab_current
                break            
            else:
                # look ahead        
                roma_ahead = roma_exp[current_idx + 1]
                arab_ahead = rome_num[roma_ahead]

                if arab_ahead <= arab_current:
                    # lookahead smaller or eq
                    total += arab_current
                    current_idx += 1
                else:
                    # lookahead bigger -> apply substruction rule
                    if can_be_substructed(roma_current, roma_ahead):
                        total += (arab_ahead - arab_current)
                        current_idx += 2
                    else:
                        raise Exception('{} cannot be substructed from {}'.format(arab_current, arab_ahead))
                        
        return total, None
    
    except Exception as ex:
        return None, ex.message
    

In [3]:
# some tests 

# ok
assert(can_be_substructed('I', 'V'))
assert(can_be_substructed('I', 'D') is False)

# ok
arabic, err = calc_roma_exp('MCMXLIV')
assert(arabic == 1944)
assert(err is None)

# ok
arabic, err = calc_roma_exp('XXXI')
assert(err is None)

# repeated error
arabic, err = calc_roma_exp('XXXXI')
assert(err is not None)

# substruct error
arabic, err = calc_roma_exp('XXIL')
assert(err is not None)

/Users/dmitry/workspace/tensorflow-sandbox/env/lib/python2.7/site-packages/ipykernel/__main__.py:100: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6


In [4]:
# parser

In [9]:
galaxy_currency_dic = {}
galaxy_currency_r = "([\w]+)[\s]+is[\s]+([IVXLCDM]+)"

# return None if not match
def parse_galaxy_currency_exp(exp):
    m = re.match(galaxy_currency_r, exp)
    if m is None:
        return None
    else:
        # return galaxy, roman_num
        galaxy_currency = m.groups()[0]
        roman_val = m.groups()[1]
        galaxy_currency_dic[galaxy_currency] = roman_val
        return (False, galaxy_currency, roman_val)
    
product_dic = {}
credit_assignment_r = "([\w\s]+)\s+([\w]+)\s+is\s+([\d]+)\s+Credits"

# return None if not match
def parse_credit_assignment_exp(exp):    
    m = re.match(credit_assignment_r, exp)
    if m is None:
        return None
    else:
        galaxy_currency_str, prod, cred = m.groups()
        galaxy_currency = galaxy_currency_str.split(' ')
        roman_exp = []
        for x in galaxy_currency:
            roman_num = galaxy_currency_dic.get(x)
            if roman_num is None:
                raise Exception('product {} is not defined'.format(x))
            else:
                roman_exp.append(roman_num)

        roman_exp_str = ''.join(roman_exp)
        arabic_val, err = calc_roma_exp(roman_exp_str)
        if err is not None:
            raise Exception(err)

        # compute unit cred
        prod_unit_cred = float(cred) / int(arabic_val)
        
        # store
        product_dic[prod] = prod_unit_cred
        return (False, prod, prod_unit_cred)
    
how_many_question_r = "how\s+many\s+Credits\s+is\s+([\w\s]+)\s+\?"

# return None if not match
def parse_how_many_exp(exp):

    m = re.match(how_many_question_r, exp)
    if m is None:
        return None
    
    variables = m.groups()[0].split(' ')
    galaxy_curency = []
    product_credit = None
    for var in variables:
        v = galaxy_currency_dic.get(var)
        v2 = product_dic.get(var)
        if v is not None:
            # galaxy currency var
            galaxy_curency.append(v)
        elif v2 is not None:
            if product_credit is None:
                product_credit = v2
            else:
                raise Exception('question can only has one product')
        else:
            raise Exception('{} is not defined'.format(var))

    galaxy_currency_val, err = calc_roma_exp(''.join(galaxy_curency))
    if err is not None:
        raise Exception(err)
        
    total_credit = galaxy_currency_val * product_credit

    msg = '{} is {} Credits'.format(' '.join(variables), total_credit)
    return (True, msg)

how_much_question_r = "how\s+much\s+is\s+([\w\s]+)\s+\?"

# return None if not match
def parse_how_much_exp(exp):
    m = re.match(how_much_question_r, exp)
    if m is None:
        return None

    variables = m.groups()[0].split(' ')
    galaxy_curency = []
    for var in variables:
        v = galaxy_currency_dic.get(var)
        if v is None:
            raise Exception('var {} is not defined'.format(var))
        galaxy_curency.append(v)

        # compute 
    galaxy_currency_val, err = calc_roma_exp(''.join(galaxy_curency))
    if err is not None:
        raise Exception(err)

    msg = '{} is {}'.format(' '.join(variables), galaxy_currency_val)
    return (True, msg)

# register parse functions
parse_functions = [
    parse_galaxy_currency_exp,
    parse_credit_assignment_exp,
    parse_how_much_exp,
    parse_how_many_exp
]

# answer user query
def answer_question(question):
    
    question_recognized = False
    for f in parse_functions:
        r = f(question)
        if r is not None:
            question_recognized = True
            should_print_result = r[0]
            if should_print_result:
                msg = r[1]
                print msg
            break

    if question_recognized is False:
        print 'I have no idea what you are talking about'


In [11]:
questions = [
    'glob is I',
    'prok is V',
    'pish is X',
    'tegj is L',
    'glob glob Silver is 34 Credits',
    'glob prok Gold is 57800 Credits',
    'pish pish Iron is 3910 Credits',
    'how much is pish tegj glob glob ?',
    'how many Credits is glob prok Silver ?',
    'how many Credits is glob prok Gold ?',
    'how many Credits is glob prok Iron ?',
    'how much wood could a woodchuck chuck if a woodchuck could chuck wood ?'   
]

for question in questions:
    answer_question(question)

pish tegj glob glob is 42
glob prok Silver is 68.0 Credits
glob prok Gold is 57800.0 Credits
glob prok Iron is 782.0 Credits
I have no idea what you are talking about
